In [1]:
import numpy as np
import pandas as pd
import random
import itertools 
import os
from collections import defaultdict
import json
import random
import numpy as np
from time import time
from tqdm import tqdm

In [2]:
#creating smaller simulations .npy file
red = ['0'] * 26
black = ['1'] * 26
deck = black + red

np.random.seed(123)

results = []
for i in tqdm(range(10000)):
    np.random.shuffle(deck)
    results += [int(''.join(deck),2)]

np.save('results_10000', results)

100%|█████████████████████████████████| 10000/10000 [00:00<00:00, 226633.38it/s]


In [3]:
#load in list of simulations, convert ot binary
def load_process_simulations(file_path): 
    simulations = np.load(file_path)
    
    def int_to_bit(n):
        binary = bin(n)[2:] #what? 
        return binary.zfill(52)
    
    binary_simulations = [int_to_bit(sim) for sim in simulations]             
    return binary_simulations

In [4]:
#for testing
processed_sims = load_process_simulations('results_10000.npy')

In [5]:
def variation1(deck, player1_sequence, player2_sequence):
    player1_cards = 0
    player2_cards = 0
    pile = 0
    
    for i in range(len(deck) - 2):
        current_sequence = deck[i:i+3]
        pile += 1
        if current_sequence == player1_sequence:
            player1_cards += pile
            pile = 0
        elif current_sequence == player2_sequence:
            player2_cards += pile
            pile = 0 
    return player1_cards, player2_cards

In [6]:
def variation2(deck, player1_sequence, player2_sequence):
    player1_tricks = 0
    player2_tricks = 0
    
    for i in range(len(deck) - 2):
        current_sequence = deck[i:i+3]
        if current_sequence == player1_sequence:
            player1_tricks += 1
        elif current_sequence == player2_sequence:
            player2_tricks += 1
    return player1_tricks, player2_tricks

In [7]:
def analyze_all_combinations(processed_sims):
    all_sequences = [''.join(seq) for seq in itertools.product('01', repeat=3)]
    results1 = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
        for p1 in all_sequences for p2 in all_sequences if p1 != p2}
    results2 = {(p1, p2): {'player1_wins': 0, 'player2_wins': 0}
        for p1 in all_sequences for p2 in all_sequences if p1 != p2}
    
    for game in processed_sims:
        for (p1_seq, p2_seq), result in results1.items():
            p1_cards, p2_cards = variation1(game, p1_seq, p2_seq)
            if p1_cards > p2_cards:
                result['player1_wins'] += 1
            elif p2_cards > p1_cards:
                result['player2_wins'] += 1

    for game in processed_sims:
        for (p1_seq, p2_seq), result in results2.items():
            p1_cards, p2_cards = variation2(game, p1_seq, p2_seq)
            if p1_cards > p2_cards:
                result['player1_wins'] += 1
            elif p2_cards > p1_cards:
                result['player2_wins'] += 1
    return results1, results2

In [8]:
results1, results2 = analyze_all_combinations(processed_sims)

In [9]:
df1 = pd.DataFrame.from_dict(results1, orient = 'index')
df1.reset_index(inplace=True)
df1.columns = ['Sequence 1','Sequence 2', 'Player 1 Wins', 'Player 2 Wins']
df1.head()


,Sequence 1,Sequence 2,Player 1 Wins,Player 2 Wins
0,000,001,4789,4987
1,000,010,2889,6907
2,000,011,1891,7842
3,000,100,3,9997
4,000,101,2687,7075


In [10]:
df2 = pd.DataFrame.from_dict(results2, orient = 'index')
df2.reset_index(inplace=True)
df2.columns = ['Sequence 1','Sequence 2', 'Player 1 Wins', 'Player 2 Wins']
df2.head()

,Sequence 1,Sequence 2,Player 1 Wins,Player 2 Wins
0,000,001,3664,5090
1,000,010,3989,4992
2,000,011,3364,5179
3,000,100,3672,5087
4,000,101,3817,4811


In [11]:
df1.to_csv('Variation1.csv', index=False)

df2.to_csv('Variation2.csv', index=False)